In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("Lucky_day1.csv")

In [3]:
df

,Date,Number 1,Number 2,Number 3,Number 4,Number 5,Number 6,Number 7,Number 8,Number 9,...,Number 11,Number 12,Number 13,Number 14,Number 15,Number 16,Number 17,Number 18,Number 19,Number 20
0,2023-07-23,1,3,8,17,19,21,31,35,41,...,48,50,66,67,68,69,70,72,76,79
1,2023-07-22,6,13,26,31,33,35,37,39,41,...,46,49,51,52,59,60,62,69,77,80
2,2023-07-21,2,3,5,7,9,14,17,24,30,...,39,49,54,58,60,62,67,68,70,72
3,2023-07-20,2,12,13,14,20,25,32,39,40,...,42,48,50,53,59,71,74,75,79,80
4,2023-07-19,3,11,13,15,20,30,33,39,41,...,50,54,57,61,64,72,74,75,77,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696,2013-01-07,3,4,8,10,18,19,20,28,29,...,45,47,48,50,55,67,69,72,77,79
3697,2013-01-05,1,2,7,8,17,23,24,30,34,...,38,48,49,53,56,64,65,66,70,73
3698,2013-01-04,4,7,9,14,19,20,21,25,28,...,34,35,40,44,60,65,72,73,75,79
3699,2013-01-03,5,7,15,16,23,27,28,30,31,...,41,43,46,48,54,65,67,69,76,78


In [4]:
new_df=df[["Number 1","Number 2","Number 3","Number 4","Number 5","Number 6","Number 7","Number 8","Number 9","Number 10","Number 11","Number 12","Number 13","Number 14","Number 15","Number 16","Number 17","Number 18","Number 19","Number 20"]]

In [5]:
scaler=StandardScaler().fit(new_df.values)

In [6]:
transformed_dataset=scaler.transform(new_df.values)

In [7]:
fransformed_df=pd.DataFrame(data=transformed_dataset, index=new_df.index)

In [8]:
number_of_row=new_df.shape[0]
window_length=7
number_of_features=new_df.values.shape[1]

###Creating train dataset and labels for each row

In [9]:
train=np.empty([number_of_row-window_length,window_length,number_of_features],dtype=float)

label =np.empty([number_of_row-window_length,number_of_features],dtype=float)

window_length=7

for i in range(0,number_of_row-window_length):
  train[i]=fransformed_df.iloc[i:i+window_length,0:number_of_features]
  label[i]=fransformed_df.iloc[i+window_length:i+window_length+1 ,0:number_of_features]

In [10]:
train.shape

(3694, 7, 20)

In [11]:
label.shape

(3694, 20)

##lstm final

In [12]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Bidirectional,Dropout

2023-07-24 21:48:19.178087: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 21:48:19.236538: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 21:48:19.237309: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 21:48:20.156119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
# model=Sequential()
# model.add(Bidirectional(LSTM(240,input_shape=(window_length,number_of_features),
#                                  return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(240,input_shape=(window_length,number_of_features),
#                                  return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(240,input_shape=(window_length,number_of_features),
#                                  return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(240,input_shape=(window_length,number_of_features),
#                                  return_sequences=True)))
# model.add(Bidirectional(LSTM(240,input_shape=(window_length,number_of_features),
#                                  return_sequences=False)))
# model.add(Dense(59))
# model.add(Dense(number_of_features))
# model.compile(loss="mse",optimizer='rmsprop', metrics=['accuracy'])

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined 'window_length', 'number_of_features', and 'output_shape'

model = Sequential()
model.add(Bidirectional(LSTM(512, input_shape=(window_length, number_of_features), return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(68, return_sequences=False)))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))  # Add a ReLU activation for better non-linearity
model.add(Dense(number_of_features, activation='linear'))  # Linear activation for regression task

# Compile the model with a suitable optimizer
optimizer = 'adam'  # 'adam' is generally a good choice for optimization
model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])  # Using 'mae' for more meaningful metric

# Implement early stopping to prevent overfitting and save training time
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)


2023-07-24 21:48:21.113947: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:

model.fit(train,label,batch_size=100,epochs=300)


Epoch 1/300


2023-07-24 21:48:22.219882: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-24 21:48:22.221864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-24 21:48:22.223316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-07-24 21:48:24.023059: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-24 21:48:24.024899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-24 21:48:24.026356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-07-24 21:48:25.864326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-24 21:48:25.866909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-24 21:48:25.869190: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-07-24 21:48:30.341937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-07-24 21:48:30.399535: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-24 21:48:30.400955: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

2023-07-24 21:48:31.946333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-07-24 21:48:32.003985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-24 21:48:32.005465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

37/37 [==============================] - 23s 171ms/step - loss: 1.0605 - mae: 0.7923
Epoch 2/300
37/37 [==============================] - 6s 167ms/step - loss: 0.9776 - mae: 0.7646
Epoch 3/300
37/37 [==============================] - 6s 172ms/step - loss: 0.9638 - mae: 0.7590
Epoch 4/300
37/37 [==============================] - 6s 174ms/step - loss: 0.9710 - mae: 0.7591
Epoch 5/300
37/37 [==============================] - 6s 173ms/step - loss: 0.9773 - mae: 0.7626
Epoch 6/300
37/37 [==============================] - 7s 188ms/step - loss: 0.9574 - mae: 0.7569
Epoch 7/300
37/37 [==============================] - 7s 179ms/step - loss: 0.9583 - mae: 0.7573
Epoch 8/300
37/37 [==============================] - 6s 175ms/step - loss: 0.9549 - mae: 0.7564
Epoch 9/300
37/37 [==============================] - 7s 184ms/step - loss: 0.9537 - mae: 0.7555
Epoch 10/300
37/37 [==============================] - 7s 179ms/step - loss: 0.9472 - mae: 0.7535
Epoch 11/300
37/37 [==============================

37/37 [==============================] - 7s 190ms/step - loss: 0.0578 - mae: 0.1893
Epoch 86/300
37/37 [==============================] - 7s 193ms/step - loss: 0.0559 - mae: 0.1853
Epoch 87/300
37/37 [==============================] - 7s 183ms/step - loss: 0.0581 - mae: 0.1893
Epoch 88/300
37/37 [==============================] - 7s 180ms/step - loss: 0.0550 - mae: 0.1847
Epoch 89/300
37/37 [==============================] - 6s 174ms/step - loss: 0.0516 - mae: 0.1780
Epoch 90/300
37/37 [==============================] - 7s 179ms/step - loss: 0.0529 - mae: 0.1808
Epoch 91/300
37/37 [==============================] - 7s 176ms/step - loss: 0.0521 - mae: 0.1795
Epoch 92/300
37/37 [==============================] - 7s 177ms/step - loss: 0.0517 - mae: 0.1788
Epoch 93/300
37/37 [==============================] - 7s 188ms/step - loss: 0.0516 - mae: 0.1787
Epoch 94/300
37/37 [==============================] - 7s 193ms/step - loss: 0.0500 - mae: 0.1760
Epoch 95/300
37/37 [=======================

In [ ]:
"""
import pickle
file_path = '/content/drive/MyDrive/lottery prediction/LSTM_final.pkl'

# Save the model using pickle
with open(file_path, 'wb') as file:
    pickle.dump(model, file)
    
    """

import pickle
file_path = 'LSTM_final.pkl'

# Save the model using pickle
with open(file_path, 'wb') as file:
    pickle.dump(model, file)

##Test

In [ ]:
df.head(15)

In [ ]:
l = []
for index, row in df.iterrows():
    l2 = []
    for i in range(1, 21):  # Assuming "Number 1" to "Number 15" are column names
        l2.append(row[f"Number {i}"])
    l.append(l2)

In [ ]:
print(l[1:8] )
print(len(l[1:8]))

In [ ]:
import pickle

file_path = 'LSTM_final.pkl'

# Load the saved model using pickle
with open(file_path, 'rb') as file:
    modell = pickle.load(file)

In [ ]:
to_predict=np.array( l[1:8] )
scaled_to_predict=scaler.transform(to_predict)

In [ ]:
#prediction

scaled_predicted_output_1=modell.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

In [ ]:
#original
l[0]

In [ ]:
#prediction
to_predict=np.array( l[2:9] )
scaled_to_predict=scaler.transform(to_predict)
scaled_predicted_output_1=modell.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

In [ ]:
#original
l[1]

In [ ]:
#prediction
to_predict=np.array( l[3:10] )
scaled_to_predict=scaler.transform(to_predict)
scaled_predicted_output_1=modell.predict(np.array([scaled_to_predict]))
pred=scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0]
print(len(pred))

In [ ]:
#original
l[2]